In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import glob
import time
import os
from utils import *

In [4]:
calibration_imgs = glob.glob("camera_cal/calibration*.jpg")
ret, mtx, dist = calibrate_cam(calibration_imgs)
print(ret)

0.921982464824774


In [11]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [157]:
def threshold_process_verbose(img, img_name, ksize=7, s_thresh=(180, 255), sx_thresh=(25, 255), sy_thresh=(15, 255), mag_thresh=(80, 255), dir_thresh=(0.7, 1.3)):
    img = np.copy(img)
    hls_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    # individual color channels:
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    h_channel = hls_img[:,:,0]
    l_channel = hls_img[:,:,1]
    s_channel = hls_img[:,:,2]
    
    v_channel = hsv_img[:,:,2]
    
    gradx = abs_sobel_thresh(l_channel,'x', ksize, sx_thresh)
    grady = abs_sobel_thresh(l_channel,'y', ksize, sy_thresh)
    mag_binary = mag_threshold(gray_img, ksize, mag_thresh)
    
    dir_binary = dir_threshold(gray_img, 7, dir_thresh)
    #print("dir_binary.shape", dir_binary.shape)
    #print(dir_binary, "\n")
    
    s_dir_blur = gaussian_blur(dir_binary, 11)
    #print("s_dir_blur.shape", s_dir_blur.shape)
    #print(s_dir_blur, "\n")
    
    dir_binary_blurred = color_threshold(s_dir_blur*255, (220,255))
    #print("dir_binary_blurred.shape", dir_binary_blurred.shape)
    #print(dir_binary_blurred, "\n")
    
    c_chan_binary = color_threshold(s_channel, s_thresh)
    
    h_chan_binary = color_threshold(h_channel, (10,40))
    
    # combine sobel masks
    gradxy = combine_masks(gradx, grady, "and")
    
    # combine sobel and color masks
    grad_and_col= combine_masks(gradxy, c_chan_binary, "or")
    
    #combine with sobel dir
    combined = combine_masks(grad_and_col, dir_binary_blurred, "or")
    
    # save to disk
    
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/01_" + img_name[12:-4] + "_HLS.jpg", hls_img)
    h_chan_img = np.dstack((h_channel, h_channel, h_channel))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/02_" + img_name[12:-4] + "_h_chan.jpg", h_chan_img)
    h_bin_img = np.dstack((h_chan_binary, h_chan_binary, h_chan_binary))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/03_" + img_name[12:-4] + "_h_binary.jpg", h_bin_img)
    
    l_chan_img = np.dstack((l_channel, l_channel, l_channel))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/04_" + img_name[12:-4] + "_l_chan.jpg", h_chan_img)
    #h_bin_img = np.dstack((h_chan_binary, h_chan_binary, h_chan_binary))*255
    #cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/05_" + img_name[12:-4] + "_h_binary.jpg", h_bin_img)
    
    s_chan_img = np.dstack((s_channel, s_channel, s_channel))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/05_" + img_name[12:-4] + "_s_chan.jpg", s_chan_img)
    #h_bin_img = np.dstack((h_chan_binary, h_chan_binary, h_chan_binary))*255
    #cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/05_" + img_name[12:-4] + "_h_binary.jpg", h_bin_img)
    
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/06_" + img_name[12:-4] + "_HSV.jpg", hsv_img)
    v_chan_img = np.dstack((v_channel, v_channel, v_channel))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/07_" + img_name[12:-4] + "_v_chan.jpg", v_chan_img)
    
    s_gradx_img = np.dstack((gradx, gradx, gradx))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/10_" + img_name[12:-4] + "_s_gradx.jpg", s_gradx_img)
    s_grady_img = np.dstack((grady, grady, grady))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/11_" + img_name[12:-4] + "_s_grady.jpg", s_grady_img)
    s_gradxy_img = np.dstack((gradxy, gradxy, gradxy))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/12_" + img_name[12:-4] + "_s_gradxy.jpg", s_gradxy_img)
    s_mag_img = np.dstack((mag_binary, mag_binary, mag_binary))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/13_" + img_name[12:-4] + "_s_mag.jpg", s_mag_img)
    s_dir_img = np.dstack((dir_binary, dir_binary, dir_binary))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/14_" + img_name[12:-4] + "_s_dir.jpg", s_dir_img)
    
    s_dir_blur_img = np.dstack((s_dir_blur, s_dir_blur, s_dir_blur))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/15_" + img_name[12:-4] + "_s_dir_blur.jpg", s_dir_blur_img)
    s_dir_binary_blurred_img = np.dstack((dir_binary_blurred, dir_binary_blurred, dir_binary_blurred))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/16_" + img_name[12:-4] + "_s_dir_binary_blurred.jpg", s_dir_binary_blurred_img)

    
    c_chan_img = np.dstack((c_chan_binary, c_chan_binary, c_chan_binary))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/17_" + img_name[12:-4] + "_c_chan.jpg", c_chan_img)
    combined_img = np.dstack((combined , combined , combined ))*255
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/18_" + img_name[12:-4] + "_combined.jpg", combined_img)
    
    return combined

In [158]:
def find_lines(img, img_name, nwindows=9, margin=110, minpix=50):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 #conversion for meters per pixel in y dimension
    xm_per_pix = 3.7/700 #conversion for meters per pixel in x dimension
    
    
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    lines_img = threshold_process_verbose(undistorted_img, img_name)
    warped_img = np.dstack((lines_img , lines_img , lines_img))*255
    binary_warped, M, Minv = warp(lines_img)
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result

    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updatedl ater for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on avg positions
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    # Concatenate the arrays of indices()
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Fit a second order polynomial to each using `np.polyfit` in meters!
    left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    return (left_fit, right_fit, left_fit_m, right_fit_m, left_lane_inds, right_lane_inds, out_img, nonzerox, nonzeroy, M, Minv)

In [159]:
def calc_Curve(y_eval, left_fit_cr, right_fit_cr):
    """
    Returns the curvature of the polynomial `fit` on the y range `yRange`.
    """
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    #res = ((1 + (2*left_fit_cr[0]*yRange*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    return left_curverad, right_curverad


In [160]:
def draw_lane_line(img, left_fit, right_fit, Minv):
    """
    Draw the lane lines on the image `img` using the poly `left_fit` and `right_fit`.
    """
    yMax = img.shape[0]
    ploty = np.linspace(0, yMax - 1, yMax)
    color_warp = np.zeros_like(img).astype(np.uint8)
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    return cv2.addWeighted(img, 1, newwarp, 0.3, 0)

In [161]:
def add_info_txt(img, left_fit_m, right_fit_m, left_curvature, right_curvature):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Calc vehicle pos
    xMax = img.shape[1]*xm_per_pix
    yMax = img.shape[0]*ym_per_pix
    vehicle_centre = xMax / 2
    line_l = left_fit_m[0]*yMax**2 + left_fit_m[1]*yMax + left_fit_m[2]
    line_r = right_fit_m[0]*yMax**2 + right_fit_m[1]*yMax + right_fit_m[2]
    line_m = line_l + (line_r - line_l)/2
    
    left_curve_txt = ("Left curvature: {0:.2f} m ").format(left_curvature)
    right_curve_txt = ("Right curvature: {0:.2f} m ").format(right_curvature)
    
    #diffFromVehicle = lineMiddle - vehicleCenter
    #if diffFromVehicle > 0:
    #    message = '{:.2f} m right'.format(diffFromVehicle)
    #else:
    #    message = '{:.2f} m left'.format(-diffFromVehicle)
        
    veh_lateral_offset = line_m - vehicle_centre #assumption is camera is centrally mounted on vehicle
    if veh_lateral_offset <= 0:
        veh_lateral_offset = abs(veh_lateral_offset)
        offset_txt = ("Vehicle is {0:.2f} m left of centre").format(veh_lateral_offset)
    else:
        offset_txt = ("Vehicle is {0:.2f} m right of centre").format(veh_lateral_offset)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (255, 255, 255)
    cv2.putText(img, left_curve_txt, (50, 50), font, 2, fontColor, 2)
    cv2.putText(img, right_curve_txt, (50, 120), font, 2, fontColor, 2)
    cv2.putText(img, offset_txt, (50, 190), font, 2, fontColor, 2)
    return img

In [162]:
test_imgs = glob.glob("test_images/*.jpg")
for img_name in test_imgs:
    if not os.path.exists("./test_images_output/" + img_name[12:-4]):
        os.makedirs("./test_images_output/" + img_name[12:-4])
    print(img_name[12:])
    img = mpimg.imread(img_name)
       
    left_fit, right_fit, left_fit_m, right_fit_m, left_lane_inds, right_lane_inds, out_img, nonzerox, nonzeroy,M, Minv = find_lines(img, img_name)
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/30_" + img_name[12:-4] + "_out_img.jpg", out_img)
    
    left_curvature, right_curvature = calc_Curve(np.max(ploty), left_fit, right_fit)
    
    print("Curvature:", left_curvature, right_curvature, "\n")
    
    lines_drawn_img = draw_lane_line(img, left_fit, right_fit, Minv)
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/31_" + img_name[12:-4] + "_lines_drawn_img.jpg", cv2.cvtColor(lines_drawn_img, cv2.COLOR_BGR2RGB))

    annotated_img = add_info_txt(lines_drawn_img, left_fit_m, right_fit_m, left_curvature, right_curvature)
    cv2.imwrite("./test_images_output/" + img_name[12:-4] +"/32_" + img_name[12:-4] + "_annotated_img.jpg", cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
print("done")

1560764250.133218.jpg
Curvature: 1049.43734004 1053.41393926 

1560764252.104561.jpg
Curvature: 2963.65961118 2106.38333236 

1560764251.549289.jpg
Curvature: 693.567947263 2175.35263696 

1560764249.125523.jpg
Curvature: 1983.59683869 1955.08181687 

1560764249.707109.jpg
Curvature: 4173.34999668 1372.43408815 

1560764248.582886.jpg
Curvature: 2057.8826365 2197.45486869 

1560764252.436681.jpg
Curvature: 2961.17281724 1569.25188117 

1560764247.492835.jpg
Curvature: 11993.8985167 1438.89246299 

1560764251.475577.jpg
Curvature: 1384.89689283 5252.33249052 

1560764245.636817.jpg
Curvature: 1245.62023797 1576.68942486 

1560764249.568771.jpg
Curvature: 2285.33599456 899.88201338 

1560764250.495607.jpg
Curvature: 2521.75130835 8352.01898864 

1560764248.97157.jpg
Curvature: 1081.37758377 1800.29354199 

1560764245.745026.jpg
Curvature: 814.650082118 1585.13871851 

1560764249.880915.jpg
Curvature: 4239.53326906 15202.7910931 

1560764251.153368.jpg
Curvature: 19456.3835217 1569.872671